In [137]:
import os
import json
import pandas as pd
import traceback

In [138]:
from langchain import HuggingFaceHub, LLMChain, PromptTemplate, HuggingFacePipeline
from langchain.chains import SequentialChain
from langchain.chains import SequentialChain
import PyPDF2
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM
import torch


In [139]:
KEY=os.getenv("HUGGINGFACEHUB_API_TOKEN")

In [ ]:
from dotenv import main

main.load_dotenv()

In [141]:
# repo_id="nvidia/Llama-3.1-Nemotron-70B-Instruct-HF"
repo_id="google/flan-t5-large"
# repo_id="meta-llama/Meta-Llama-3-8B-Instruct"
model_id=repo_id

In [142]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [143]:
TEMPLATE="""
Text{text}
You are a expert MCQ maker. Given the above test, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the question are not repeated and checks all the questions to be conforming the text as well.
Make sure to format your responsse like RESPONSE_JSON below adn use it as guide. \
Ensure to make {number} MCQS
*** RESPONSE_JSON
{response_json}
"""

In [144]:
quiz_generation_prompt = PromptTemplate(
    input_variables=['text', 'number', 'subject', "tone", 'response_json'],
    template=TEMPLATE
)

In [145]:
llm=HuggingFaceHub

In [146]:
quiz_chain = LLMChain(llm=llm(repo_id=repo_id, model_kwargs={'temperature':0}), prompt=quiz_generation_prompt, output_key='quiz', verbose=True)

In [147]:
TEMPLATE2="""
You are an expert english gramarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the questions and give a complete analysis of the quiz. Only use at max 50 words for complexity if the quiz is not at per with the congitive and analytical abilities of the students.\
Update the quiz questions which needs to be changed the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert  english writer of the above quiz:
"""

In [148]:
quiz_evaluation_prompt=PromptTemplate(
    input_variables=["subject", "quiz"],
    template=TEMPLATE2
)

In [149]:
review_chain = LLMChain(llm=llm(repo_id=repo_id, model_kwargs={'temperature':0}), prompt=quiz_evaluation_prompt, output_key='review', verbose=True)

In [150]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)


In [151]:
file_path=r"../data.txt"

In [ ]:
file_path


In [153]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [ ]:
print(TEXT)

In [ ]:
# Serialize  the Python dictionary into a JSON-formatted string

json.dumps(RESPONSE_JSON)

In [158]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import json

class TokenTracker:
    def __init__(self):
        self.input_tokens = 0
        self.output_tokens = 0

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        print(f"Total Input Tokens: {self.input_tokens}")
        print(f"Total Output Tokens: {self.output_tokens}")

    def add_input_tokens(self, count):
        self.input_tokens += count

    def add_output_tokens(self, count):
        self.output_tokens += count

# Load the tokenizer and model
model_id = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)


In [162]:
# Example usage of the function
# TEXT = "Translate this text to german."
NUMBER = 5
SUBJECT = "Machine Learning"
TONE = "simple"
# RESPONSE_JSON = {"example_key": "example_value"}


In [ ]:
# Create input data
input_data = {
    "text": TEXT,
    "number": NUMBER,
    "subject": SUBJECT,
    "tone": TONE,
    "response_json": json.dumps(RESPONSE_JSON)
}

def process_translation(input_data):
    input_text = input_data["text"]

    # Define the maximum length for each chunk and output
    max_chunk_length = 512
    max_output_length = 512

    # Split the input text into smaller chunks
    input_chunks = [input_text[i:i+max_chunk_length] for i in range(0, len(input_text), max_chunk_length)]

    responses = []
    with TokenTracker() as tracker:
        for chunk in input_chunks:
            # Tokenize the chunk
            input_tokens = tokenizer.encode(chunk, return_tensors="pt", max_length=max_chunk_length, truncation=True)
            input_token_count = len(input_tokens[0])
            tracker.add_input_tokens(input_token_count)

            # Generate output based on the chunk tokens
            output_tokens = model.generate(input_tokens, max_length=max_output_length)
            output_token_count = len(output_tokens[0])
            tracker.add_output_tokens(output_token_count)

            # Decode the output tokens to get the response text for the chunk
            response_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
            responses.append(response_text)

    # Combine all the responses
    final_response = " ".join(responses)
    return final_response

# Get the response for the input data
response_text = process_translation(input_data)

# Display the response
print(f"Response Text: {response_text}")


In [ ]:
generate_response(TEXT, NUMBER, SUBJECT, TONE, RESPONSE_JSON)

In [168]:
import os
import json
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# TokenTracker class to track input and output tokens
class TokenTracker:
    def __init__(self):
        self.input_tokens = 0
        self.output_tokens = 0

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        print(f"Total Input Tokens: {self.input_tokens}")
        print(f"Total Output Tokens: {self.output_tokens}")

    def add_input_tokens(self, count):
        self.input_tokens += count

    def add_output_tokens(self, count):
        self.output_tokens += count

# Load the tokenizer and model
model_id = "google/flan-t5-large"  # Example model, replace with your own
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

# Example usage of the function
def generate_response(text, number, subject, tone, response_json):
    # Check if any input data is missing
    if not all([text, number, subject, tone, response_json]):
        raise ValueError("All input parameters must be provided.")

    # Tokenize the input text
    input_tokens = tokenizer.encode(text, return_tensors="pt")

    # Define a fixed maximum length for the model's input
    max_length = 512  # You can adjust this value as needed for your model
    if input_tokens.size(1) > max_length:
        input_tokens = input_tokens[:, :max_length]
        print(f"Warning: Input truncated to {max_length} tokens.")

    with TokenTracker() as tracker:
        # Track the number of input tokens
        input_token_count = len(input_tokens[0])
        tracker.add_input_tokens(input_token_count)

        # Generate the output while handling memory errors
        try:
            output_tokens = model.generate(
                input_tokens, 
                max_length=min(15000, input_token_count + 100),  # Ensure a reasonable max length
                num_beams=5,  # Optional: Set number of beams for beam search
                early_stopping=True
            )
        except torch.cuda.OutOfMemoryError:
            raise RuntimeError("Out of memory. Try reducing the input size or use a smaller model.")

        # Track the number of output tokens
        output_token_count = len(output_tokens[0])
        tracker.add_output_tokens(output_token_count)

        # Decode the generated output
        response_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)

    return response_text

# Example data
TEXT = TEXT
NUMBER = 5
SUBJECT = "Machine Learning"
TONE = "formal"
RESPONSE_JSON = RESPONSE_JSON

# Call the function
try:
    result = generate_response(TEXT, NUMBER, SUBJECT, TONE, RESPONSE_JSON)
    print("Generated Response:", result)
except Exception as e:
    print("An error occurred:", e)


Total Input Tokens: 8
Total Output Tokens: 12
Generated Response:     


In [ ]:
# For OpenAI  
# How to setup Token Usage  Tracking  in Langchain

# with get_openai_callback() as cb:
#     response=generate_evaluate_chain(
#         {
#             "text": TEXT,
#             "number": NUMBER,
#             "subjecy": SUBJECT, 
#             "tone": TONE,
#             "respose_json": json.dumps(RESPONSE_JSON)
#         }
#     )